In [ ]:
import json
import pandas as pd


# Load data from JSON file and Convert data to pandas DataFrame
df = pd.read_json('data.json', convert_dates=['timestamp'])

In [ ]:
print(df.columns)

In [ ]:
# set timestamp as the index
df.set_index('timestamp', inplace=True)
# Sort the DataFrame by the index 
df = df.sort_index()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Convert categorical features to one-hot encoded format
df = pd.get_dummies(df, columns=['requestUrl', 'status', 'source_app','requestMethod'])

# Normalize the numerical features
scaler = MinMaxScaler()
df[["latency", "requestSize", "responseSize",]] = scaler.fit_transform(df[["latency", "requestSize", "responseSize"]])

In [ ]:
print(df.columns)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Rescale the data using the training data
scaler = StandardScaler()
train_size = int(len(df) * 0.95)
train = df[:train_size][df.columns]
test = df[train_size:][df.columns]
scaler.fit(train)
df[df.columns] = scaler.transform(df[df.columns])

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model

# Load pre-processed data
X = train

# Define the model
inputs = Input(shape=(X.shape[1], X.shape[2]))
lstm1 = LSTM(64, return_sequences=True)(inputs)
lstm2 = LSTM(32)(lstm1)
dense1 = Dense(32, activation='relu')(lstm2)
outputs = Dense(X.shape[2], activation='sigmoid')(dense1)
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='mse', optimizer='adam')

# Train the model
history = model.fit(X, X, epochs=100, batch_size=32, validation_split=0.1)


In [ ]:
# Generate predictions on test data
y_pred = model.predict(test)

# Denormalize the predicted values
y_pred_denorm = scaler.inverse_transform(y_pred)

# Calculate the mean squared error
mse = np.mean(np.square(test - y_pred_denorm))

print('Test MSE:', mse)
